In [2]:
import pandas as pd
import numpy as np
from sklearn import preprocessing
from sklearn.feature_extraction import DictVectorizer
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn import linear_model
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.cross_validation import KFold
from sklearn.metrics import mean_squared_error
import matplotlib.pyplot as plt

/usr/local/lib/python2.7/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [4]:
train=pd.read_csv('Atrain.csv')
test=pd.read_csv('Atest.csv')
submission=pd.read_csv('/home/saikantareddy/Downloads/Sample_Submission_Tm9Lura.csv')
#x=['User_ID','Product_ID','Gender','Age','Occupation','City_Category','Stay_In_Current_City_Years','Marital_Status','Product_Category_1','Product_Category_2','Product_Category_3','Purchase']
#for  i in x:
#    for dataset in combine:
#        try:
            #print(dataset[i].value_counts())
#            dataset[i].fillna(dataset[i].median(),inplace=True)
#            #dataset[i].astype('int')
#        except:
#            continue


In [5]:
combine=[train,test]
input=pd.concat(combine)
input.shape
input.head()

,Age,City_Category,Gender,Marital_Status,Occupation,Product_Category_1,Product_Category_2,Product_Category_3,Product_ID,Purchase,Stay_In_Current_City_Years,User_ID
0,0-17,A,F,0,10,3,NaN,NaN,P00069042,8370.0,2,1000001
1,0-17,A,F,0,10,1,6.0,14.0,P00248942,15200.0,2,1000001
2,0-17,A,F,0,10,12,NaN,NaN,P00087842,1422.0,2,1000001
3,0-17,A,F,0,10,12,14.0,NaN,P00085442,1057.0,2,1000001
4,55+,C,M,0,16,8,NaN,NaN,P00285442,7969.0,4+,1000002


In [6]:
input.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 783667 entries, 0 to 233598
Data columns (total 12 columns):
Age                           783667 non-null object
City_Category                 783667 non-null object
Gender                        783667 non-null object
Marital_Status                783667 non-null int64
Occupation                    783667 non-null int64
Product_Category_1            783667 non-null int64
Product_Category_2            537685 non-null float64
Product_Category_3            237858 non-null float64
Product_ID                    783667 non-null object
Purchase                      550068 non-null float64
Stay_In_Current_City_Years    783667 non-null object
User_ID                       783667 non-null int64
dtypes: float64(3), int64(4), object(5)
memory usage: 77.7+ MB


In [7]:
input.fillna(-999,inplace=True)

In [8]:
target=input['Purchase']
target=np.array(target)

In [9]:
input.drop(['Purchase'],axis=1,inplace=True)

In [10]:
input=input.applymap(str)
input.dtypes

Age                           object
City_Category                 object
Gender                        object
Marital_Status                object
Occupation                    object
Product_Category_1            object
Product_Category_2            object
Product_Category_3            object
Product_ID                    object
Stay_In_Current_City_Years    object
User_ID                       object
dtype: object

In [11]:
input_pd=input.copy()

In [12]:
input=np.array(input)
for i in range(input.shape[1]):
    lbl=preprocessing.LabelEncoder()
    lbl.fit(list(input[:,i]))
    input[:,i]=lbl.transform(input[:,i])

In [13]:
input=input.astype(int)

In [14]:
first_stage_rows=np.random.randint(train.shape[0],size=np.int(train.shape[0]/2))
first_stage_rows.shape

(275034,)

In [15]:
train_np=input[:train.shape[0],:]
target_np=target[:train.shape[0]]
train_fs=train_np[first_stage_rows,:]
target_fs=target_np[first_stage_rows]
train_ss=train_np[-first_stage_rows,:]
target_ss=target_np[-first_stage_rows]

In [16]:
xgtrain=xgb.DMatrix(train_fs,label=target_fs)
watchlist=[(xgtrain,'train')]
params={}
params['min_child_weight']=10
params['subsample']=0.7
params['colsample_bytree']=0.7
params['scale_pos_weight']=0.8
params['max_depth']=6
params['nthread']=6
params['objective']='reg:linear'
params['eta']=0.1
params['base_score']=1800
params['eval_metric']='rmse'
params['seed']=0

plst=list(params.items())
num_rounds=3000
model_1=xgb.train(plst,xgtrain,num_rounds)

params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 8
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 1420

model_2 = xgb.train(plst, xgtrain, num_rounds)

# Model 3: 10/1200

params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 10
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 1200

model_3 = xgb.train(plst, xgtrain, num_rounds)

# Model 4: 12/800

params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 12
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 800

model_4 = xgb.train(plst, xgtrain, num_rounds)

In [17]:
model_5 = ExtraTreesRegressor(n_estimators=1450, 
                              max_depth=8,
                              min_samples_split=10, 
                              min_samples_leaf=10, 
                              oob_score=True, 
                              n_jobs=6, 
                              random_state=123, 
                              verbose=1, 
                              bootstrap=True)
model_5.fit(train_fs, target_fs)

# Model 6: 6/3000

model_6 = ExtraTreesRegressor(n_estimators=3000, 
                              max_depth=6,
                              min_samples_split=10, 
                              min_samples_leaf=10, 
                              oob_score=True, 
                              n_jobs=6, 
                              random_state=123, 
                              verbose=1, 
                              bootstrap=True)
model_6.fit(train_fs, target_fs)

# Model 7: 12/800

model_7 = ExtraTreesRegressor(n_estimators=800, 
                              max_depth=12,
                              min_samples_split=10, 
                              min_samples_leaf=10, 
                              oob_score=True, 
                              n_jobs=6, 
                              random_state=123, 
                              verbose=1, 
                              bootstrap=True)
model_7.fit(train_fs, target_fs)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    6.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   29.8s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  1.1min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  2.1min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  3.3min
[Parallel(n_jobs=6)]: Done 1450 out of 1450 | elapsed:  3.9min finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    5.7s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   25.4s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:  1.0min
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:  1.9min
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:  3.3min
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:  4.6min
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:  6.2min
[Parallel(n_jobs=6)]: Done 3000 out of 3000 | elapsed:  7.5min finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    7.6s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:   36.4

ExtraTreesRegressor(bootstrap=True, criterion='mse', max_depth=12,
          max_features='auto', max_leaf_nodes=None,
          min_impurity_decrease=0.0, min_impurity_split=None,
          min_samples_leaf=10, min_samples_split=10,
          min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=6,
          oob_score=True, random_state=123, verbose=1, warm_start=False)

In [18]:
model_8 = RandomForestRegressor(n_estimators=3000, max_depth=6, oob_score=True, n_jobs=6, random_state=123, min_samples_split=10, min_samples_leaf=10)
model_8.fit(train_fs, target_fs)

# Model 9: 8/1500
model_9 = RandomForestRegressor(n_estimators=1500, max_depth=8, oob_score=True, n_jobs=6, random_state=123, min_samples_split=10, min_samples_leaf=10)
model_9.fit(train_fs, target_fs)

# Model 10: 12/800
model_10 = RandomForestRegressor(n_estimators=800, max_depth=12, oob_score=True, n_jobs=6, random_state=123, min_samples_split=10, min_samples_leaf=10)
model_10.fit(train_fs, target_fs)

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=12,
           max_features='auto', max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=10, min_samples_split=10,
           min_weight_fraction_leaf=0.0, n_estimators=800, n_jobs=6,
           oob_score=True, random_state=123, verbose=0, warm_start=False)

In [19]:
model_1_predict = model_1.predict(xgb.DMatrix(train_ss))
model_2_predict = model_2.predict(xgb.DMatrix(train_ss))
model_3_predict = model_3.predict(xgb.DMatrix(train_ss))
model_4_predict = model_4.predict(xgb.DMatrix(train_ss))
model_5_predict = model_5.predict(train_ss)
model_6_predict = model_6.predict(train_ss)
model_7_predict = model_7.predict(train_ss)
model_8_predict = model_8.predict(train_ss)
model_9_predict = model_9.predict(train_ss)
model_10_predict = model_10.predict(train_ss)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.7s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    1.7s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    3.0s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    4.7s
[Parallel(n_jobs=6)]: Done 1450 out of 1450 | elapsed:    5.5s finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    1.3s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    2.4s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    3.7s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    5.4s
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:    7.3s
[Parallel(n_jobs=6)]: Done 3000 out of 3000 | elapsed:    9.0s finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    1.3

In [20]:
train_ss_w_meta = np.concatenate((train_ss, np.vstack((model_1_predict, model_2_predict, model_3_predict, 
                                                       model_4_predict, model_5_predict,
              model_6_predict, model_7_predict, model_8_predict, model_9_predict, model_10_predict)).T), axis=1)

In [21]:
kfolds = KFold(train_ss_w_meta.shape[0], n_folds=5)

In [22]:
params = {}
params["min_child_weight"] = 10
params["subsample"] = 0.7
params["colsample_bytree"] = 0.7
params["scale_pos_weight"] = 0.8
params["silent"] = 1
params["max_depth"] = 8
params["nthread"] = 6
#params["gamma"] = 1
params["objective"] = "reg:linear"
params["eta"] = 0.1
params["base_score"] = 1800
params["eval_metric"] = "rmse"
params["seed"] = 0

plst = list(params.items())
num_rounds = 1400

In [23]:
for train_index, validation_index in kfolds:
    
    train_X, validation_X = train_ss_w_meta[train_index, :], train_ss_w_meta[validation_index, :]
    train_y, validation_y = target_ss[train_index], target_ss[validation_index]
    
    xgtrain = xgb.DMatrix(train_X, label=train_y)
    watchlist = [(xgtrain, 'train')]
    model_cv_xgboost = xgb.train(plst, xgtrain, num_rounds)
    model_cv_predict = model_cv_xgboost.predict(xgb.DMatrix(validation_X))
    print(np.sqrt(mean_squared_error(validation_y, model_cv_predict)))

9025.704073015733
9000.82897924893
9019.842168511079
8984.081057545223
9035.337397844334


In [24]:
xgtrain = xgb.DMatrix(train_ss_w_meta, label=target_ss)
watchlist = [(xgtrain, 'train')]
model_ss_xgboost = xgb.train(plst, xgtrain, num_rounds)

In [25]:
model_1_predict = model_1.predict(xgb.DMatrix(input[train.shape[0]:, :]))
model_2_predict = model_2.predict(xgb.DMatrix(input[train.shape[0]:, :]))
model_3_predict = model_3.predict(xgb.DMatrix(input[train.shape[0]:, :]))
model_4_predict = model_4.predict(xgb.DMatrix(input[train.shape[0]:, :]))
model_5_predict = model_5.predict(input[train.shape[0]:, :])
model_6_predict = model_6.predict(input[train.shape[0]:, :])
model_7_predict = model_7.predict(input[train.shape[0]:, :])
model_8_predict = model_8.predict(input[train.shape[0]:, :])
model_9_predict = model_9.predict(input[train.shape[0]:, :])
model_10_predict = model_10.predict(input[train.shape[0]:, :])

test_ss_w_meta = np.concatenate((input[train.shape[0]:, :], np.vstack((model_1_predict, model_2_predict, model_3_predict, 
                                                       model_4_predict, model_5_predict,
              model_6_predict, model_7_predict, model_8_predict, model_9_predict, model_10_predict)).T), axis=1)

[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.1s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    1.4s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    2.6s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    4.1s
[Parallel(n_jobs=6)]: Done 1450 out of 1450 | elapsed:    4.8s finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.2s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    0.6s
[Parallel(n_jobs=6)]: Done 438 tasks      | elapsed:    1.2s
[Parallel(n_jobs=6)]: Done 788 tasks      | elapsed:    2.1s
[Parallel(n_jobs=6)]: Done 1238 tasks      | elapsed:    3.2s
[Parallel(n_jobs=6)]: Done 1788 tasks      | elapsed:    4.5s
[Parallel(n_jobs=6)]: Done 2438 tasks      | elapsed:    6.1s
[Parallel(n_jobs=6)]: Done 3000 out of 3000 | elapsed:    7.5s finished
[Parallel(n_jobs=6)]: Done  38 tasks      | elapsed:    0.3s
[Parallel(n_jobs=6)]: Done 188 tasks      | elapsed:    1.2

In [26]:
model_ss_predict = model_ss_xgboost.predict(xgb.DMatrix(test_ss_w_meta))

In [27]:
np.max(model_ss_predict), np.min(model_ss_predict)

(1800.0, 1800.0)

In [28]:
submission.Purchase = model_ss_predict

In [29]:
submission.to_csv("submit_23.csv", index=False)